## Credentials

github: gozelalmazovna

You can change the code however you want, have fun!

### Import libraries

In [1]:
import cv2
import numpy as np

In [2]:
#Modified algorithm for iou that returns True if there is no overlap else returns False
def calculate_iou(a, b, small):
    x1 = max(a[0], b[0])
    y1 = max(a[1], b[1])
    x2 = min(a[2], b[2])
    y2 = min(a[3], b[3])

    width = (x2 - x1)
    height = (y2 - y1)

    if (width<0) or (height <0):
        return 0.0
    area_overlap = width * height

    area_a = (a[2] - a[0]) * (a[3] - a[1])
    area_b = (b[2] - b[0]) * (b[3] - b[1])
    area_combined = area_a + area_b - area_overlap

    iou = area_overlap / (area_combined+small)

    if iou > 0.2:
        return True
    else:
        return False

In [3]:
#Funtion for implementation of iou for every template
def iou_implement(result,threshold,w,h):
    new_loc = []

    #Take the locations where result is bigger or equal to the threshold
    loc = np.where(result >= threshold)

    #Then we go through image iteration by part
    for pta in zip(*loc[::-1]):
        a = [pta[0],pta[1] ,pta[0] + w, pta[1] + h]

        for ptb in zip(*loc[::-1]):
            b= [ptb[0],ptb[1] ,ptb[0] + w, ptb[1] + h]

            if calculate_iou(a,b,0.0001) is True:
                if ptb not in new_loc:
                    new_loc.append(ptb)

    return new_loc

In [4]:
#Read the image
img_rgb = cv2.imread('fullImage.jpg')
img_detected = cv2.imread('fullImage.jpg')

#Convert to Grayscale for faster implementation
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

#Arrange templates
templates = [cv2.imread('component1.jpg',0),cv2.imread('component2.jpg',0),cv2.imread('component3.jpg',0)]

#Thrsholds fow every template
thresholds = [0.84,0.75,0.7]

result = []
width = []
height = []

In [5]:
#Perform template matching for every template
for template in templates:
    w, h = template.shape[::-1]
    width.append(w)
    height.append(h)
    result.append(cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED))

In [6]:
#Perform iou calculation
new_result = []
for i in range(3):
    new_result.append(iou_implement(result[i],thresholds[i],width[i],height[i]))

In [7]:
#Draw the rectangles for each component
for pt in new_result[0]:
    cv2.rectangle(img_detected, pt, (pt[0] + width[0], pt[1] + height[0]), (100,30,230), 2)

for pt in new_result[1]:
    cv2.rectangle(img_detected, pt, (pt[0] + width[1], pt[1] + height[1]), (100,230,230), 2)

for pt in new_result[2]:
    cv2.rectangle(img_detected, pt, (pt[0] + width[2], pt[1] + height[2]), (200,200,200), 2)

In [8]:
# Show the image in new window
cv2.imshow('Detected', img_detected)
cv2.waitKey(0)

-1